<h1 style="color:black" align="center">Семинар №9</h1>

<h1 style="color:#008B8B">1. Градиентный бустинг</h1>

На лекциях обсуждался градиентный бустинг - один из самых мощных методов в машинном обучении. Он позволяет воспользоваться выразительной силой решающих деревьев и при этом контролировать их переобучение. Ниже мы пройдём по основным блокам градиентного бустинга и поймём, почему они устроены именно так.

Для начала вспомним основные принципы градиентного бустинга. Будем искать алгоритм, оптимизирующий некоторую дифференцируемую функцию потерь $L(y, z)$, в виде взвешенной суммы базовых алгоритмов:

$\large a_N(x)
=
\sum\limits_{n = 0}^{N}
    \gamma_n b_n(x).$

Как правило, веса $\gamma_n$ не подбираются и полагаются равными единице (поскольку, как правило, в деревьях всё равно потом тщательно подбираются прогнозы в листьях), но пока будет рассматривать общий случай.

Идея бустинга заключается в последовательном построении алгоритмов, каждый из которых учитывает ошибки построенной до сих пор композиции:

$\large \sum\limits_{i = 1}^{\ell}
    L(y_i, a_{N - 1}(x_i) + \gamma_N b_N(x_i))
\to
\underset{b_N, \gamma_N}{\text{min}}$


После выбора каких-нибудь простых $\gamma_0$ и $b_0(x)$ (например, для задачи регрессии можно положить $\gamma_0 = 1$ и $b_0(x) = \frac 1\ell \sum\limits_{i=1}^\ell y_i$) все последующие базовые алгоритмы стараются приблизить антиградиент функционала ошибки, взятый в точках $z = a_{N - 1}(x_i)$:

$\large s_i
=
-
\left.
\frac{\partial L(y_i, z)}{\partial z}
\right|_{z = a_{N - 1}(x_i)}$


При этом приближается антиградиент с точки зрения квадратичной функции потерь:

$\large b_N(x)
=
\underset{b \in \mathbb{A}}{\text{argmin}}
    \sum\limits_{i = 1}^{\ell}
        \left(
            b(x_i) - s_i
        \right)^2$


Подбор коэффициентов производится просто через задачу одномерной оптимизации:

$\large \gamma_N
=
\underset{\gamma \in \mathbb{R}}{\text{argmin}}
\sum\limits_{i=1}^{\ell}
L(y_i, a_{N-1}(x_i) + \gamma b_N (x_i))$


Обсудим теперь подробнее некоторые моменты.

<h2 style="color:#008B8B">1.1 Почему градиентный бустинг устроен именно так?</h2>